# Decision Trees

Leitura dos datasets tratados

In [6]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re # Used in Data Processing
from sklearn import preprocessing

allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']

treino = pd.read_csv('datasets/treinoTratado.csv', na_values=allowed_nans, keep_default_na=False)
teste = pd.read_csv('datasets/testeTratado.csv', na_values=allowed_nans, keep_default_na=False)


## Construção e modelação

In [7]:


label_encoder = preprocessing.LabelEncoder() 

treino['Date']= label_encoder.fit_transform(treino['Date'])
teste['Date']= label_encoder.fit_transform(teste['Date'])

treino['Month']= label_encoder.fit_transform(treino['Month'])
teste['Month']= label_encoder.fit_transform(teste['Month'])

treino['Season']= label_encoder.fit_transform(treino['Season'])
teste['Season']= label_encoder.fit_transform(teste['Season'])

treino['weather_description']= label_encoder.fit_transform(treino['weather_description'])
teste['weather_description']= label_encoder.fit_transform(teste['weather_description'])



In [8]:
X_treino = treino.drop('Injeçao na rede (kWh)', axis=1)
y_treino = treino['Injeçao na rede (kWh)']

X_teste = teste

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_treino_normalized = scaler.fit_transform(X_treino)
X_teste_normalized = scaler.fit_transform(X_teste)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=183, max_iter=2000)

params = {
    'activation': ['relu', 'tanh', 'logistic'],
    'learning_rate_init': [0.05, 0.1, 0.3],  # Learning rate
    'hidden_layer_sizes': [(50,50), (100,100)],
}

grid = GridSearchCV(estimator=clf,
                    param_grid=params,
                    cv=10,
                    n_jobs=1,
                    verbose=2)
grid.fit(X_treino_normalized, y_treino)
grid.best_score_, grid.best_params_

Fitting 10 folds for each of 18 candidates, totalling 180 fits
[CV] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate_init=0.05; total time=   3.1s
[CV] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate_init=0.05; total time=   1.6s
[CV] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate_init=0.05; total time=   2.0s
[CV] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate_init=0.05; total time=   1.6s
[CV] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate_init=0.05; total time=   1.6s
[CV] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate_init=0.05; total time=   1.4s
[CV] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate_init=0.05; total time=   1.9s
[CV] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate_init=0.05; total time=   1.3s
[CV] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate_init=0.05; total time=   2.0s
[CV] END activation=relu, hidden_layer_sizes=(

(0.8209189009456624,
 {'activation': 'relu',
  'hidden_layer_sizes': (50, 50),
  'learning_rate_init': 0.05})

In [11]:
predictions = grid.predict(X_teste_normalized)

predict_data = {'Result': predictions}
df = pd.DataFrame(predict_data)
df.index += 1
df.to_csv(f'predictions/redesNeuronaisPrediction.csv')